# Quantum Convolutional Neural Network

This is an implementation of 8 Qubit QCNN circuit (Quantum Convolutional Neural Networks by Iris Cong et al, 2019) and Tree Tensor Network circuit (Hierarchical Quantum Classifier by Ed Grant et al, 2018) with the comparisons among various ansatze.

In [1]:
import pennylane as qml
from pennylane import numpy as np
np.random.seed(0)

# QCNN Circuit

Here are various unitary ansatze to test

In [2]:
# Unitraies for Convolutional Layers 
def U_TTN(params, wires): # 2 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])

def U_5(params, wires): # 10 params
    qml.RX(params[0], wires = wires[0])
    qml.RX(params[1], wires = wires[1])
    qml.RZ(params[2], wires = wires[0])
    qml.RZ(params[3], wires = wires[1])
    qml.CRZ(params[4], wires = [wires[1], wires[0]])
    qml.CRZ(params[5], wires = [wires[0], wires[1]])
    qml.RX(params[6], wires = wires[0])
    qml.RX(params[7], wires = wires[1])
    qml.RZ(params[8], wires = wires[0])
    qml.RZ(params[9], wires = wires[1])
    
def U_6(params, wires): # 10 params
    qml.RX(params[0], wires = wires[0])
    qml.RX(params[1], wires = wires[1])
    qml.RZ(params[2], wires = wires[0])
    qml.RZ(params[3], wires = wires[1])
    qml.CRX(params[4], wires = [wires[1], wires[0]])
    qml.CRX(params[5], wires = [wires[0], wires[1]])
    qml.RX(params[6], wires = wires[0])
    qml.RX(params[7], wires = wires[1])
    qml.RZ(params[8], wires = wires[0])
    qml.RZ(params[9], wires = wires[1])

def U_9(params, wires): # 2 params
    qml.Hadamard(wires = wires[0])
    qml.Hadamard(wires = wires[1])
    qml.CZ(wires = [wires[0],wires[1]])
    qml.RX(params[0], wires = wires[0])
    qml.RX(params[1], wires = wires[1])

def U_13(params, wires): # 6 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CRZ(params[2], wires = [wires[1], wires[0]])
    qml.RY(params[3], wires = wires[0])
    qml.RY(params[4], wires = wires[1])
    qml.CRZ(params[5], wires = [wires[0], wires[1]])

def U_14(params, wires): # 6 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CRX(params[2], wires = [wires[1], wires[0]])
    qml.RY(params[3], wires = wires[0])
    qml.RY(params[4], wires = wires[1])
    qml.CRX(params[5], wires = [wires[0], wires[1]])

def U_15(params, wires): # 4 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CNOT(wires = [wires[1], wires[0]])
    qml.RY(params[2], wires = wires[0])
    qml.RY(params[3], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])

def U_SO4(params, wires): # 6 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])
    qml.RY(params[2], wires = wires[0])
    qml.RY(params[3], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])
    qml.RY(params[4], wires = wires[0])
    qml.RY(params[5], wires = wires[1])

# Unitraies for Pooling and Fully Connected Layers
def V_0(theta, wires):
    qml.CRZ(theta, wires = [wires[0], wires[1]])

def V_1(theta, wires):
    qml.CRX(theta, wires = [wires[0], wires[1]])

def F(theta, wires):
    qml.CRZ(theta, wires = [wires[0], wires[1]])

Below is general circuit structures used in Quantum Convolutional Neural Network (QCNN) circuits.

In [3]:
# Convolution Layer1
def conv_layer1(U, params):
    U(params, wires = [0,7])
    for i in range (0,8,2):
        U(params, wires = [i, i+1])
    for i in range (1,7,2):
        U(params, wires = [i, i+1])
    
def conv_layer2(U, params):
    U(params, wires = [0,2])
    U(params, wires = [4,6])
    U(params, wires = [2,4])
    U(params, wires = [0,6])
    
def pooling_layer1(V_0, V_1, params):
    for i in range(0,8,2):
        V_0(params[0], wires = [i+1, i])
    for i in range(0,8,2):
        qml.PauliX(wires = i+1)
    for i in range(0,8,2):
        V_1(params[1], wires = [i+1, i])
        

def pooling_layer2(V_0, V_1, params): # 2params
    V_0(params[0], wires = [2,0])
    V_0(params[0], wires = [6,4])
    
    qml.PauliX(wires = 2)
    qml.PauliX(wires = 6)
    
    V_1(params[1], wires = [2,0])
    V_1(params[1], wires = [6,4])

Here we define various possible embedding methods.

In [4]:
from pennylane.templates.embeddings import AmplitudeEmbedding, AngleEmbedding

def data_embedding(X, embedding_type = 'Amplitude'):
    if embedding_type == 'Amplitude':
        AmplitudeEmbedding(X, wires = range(8), normalize = True)
    elif embedding_type == 'Angle':
        AngleEmbedding(X, wires = range(8), rotation = 'Y')

Define QCNN circuit with given Unitary Ansatz and embedding method.

In [5]:
dev = qml.device('default.qubit', wires = 8)

@qml.qnode(dev)
def QCNN(X, params, U, U_params, embedding_type = 'Amplitude'):

    param1 = params[0:U_params]
    param2 = params[U_params:U_params + 2]
    param3 = params[U_params + 2: 2*U_params + 2]
    param4 = params[2*U_params + 2: 2*U_params + 4]
    param5 = params[2*U_params + 4]
    
    # Data Embedding
    data_embedding(X, embedding_type = embedding_type)
    
    #Quantum Convolutional Neural Network
    if U == 'U_TTN':
        conv_layer1(U_TTN, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_TTN, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_5':
        conv_layer1(U_5, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_5, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_6':
        conv_layer1(U_6, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_6, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_9':
        conv_layer1(U_9, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_9, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_13':
        conv_layer1(U_13, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_13, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_14':
        conv_layer1(U_14, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_14, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_15':
        conv_layer1(U_15, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_15, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
    
    elif U == 'U_SO4':
        conv_layer1(U_SO4, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_SO4, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    return qml.expval(qml.PauliZ(4))

Below is Hierarchical Quantum Classifier structure with different Ansatze

In [6]:
dev_TTN = qml.device('default.qubit', wires = 8)

@qml.qnode(dev_TTN)
def Hierarchical_classifier(X, params, U, U_params, embedding_type = 'Amplitude'):
    
    param1 = params[0 * U_params:1 * U_params]
    param2 = params[1 * U_params:2 * U_params]
    param3 = params[2 * U_params:3 * U_params]
    param4 = params[3 * U_params:4 * U_params]
    param5 = params[4 * U_params:5 * U_params]
    param6 = params[5 * U_params:6 * U_params]
    param7 = params[6 * U_params:7 * U_params]
    
    data_embedding(X, embedding_type = embedding_type)
    ['U_TTN', 'U_5', 'U_6', 'U_13', 'U_14', 'U_15', 'U_SO4'] 
    if U == 'U_TTN':
        # layer 1
        U_TTN(param1, wires = [0,1])
        U_TTN(param2, wires = [2,3])
        U_TTN(param3, wires = [4,5])
        U_TTN(param4, wires = [6,7])
        # layer 2
        U_TTN(param5, wires = [1,3])
        U_TTN(param6, wires = [5,7])
        # layer 3
        U_TTN(param7, wires = [3,7])
    elif U == 'U_5':
        # layer 1
        U_5(param1, wires = [0,1])
        U_5(param2, wires = [2,3])
        U_5(param3, wires = [4,5])
        U_5(param4, wires = [6,7])
        # layer 2
        U_5(param5, wires = [1,3])
        U_5(param6, wires = [5,7])
        # layer 3
        U_5(param7, wires = [3,7])
    elif U == 'U_6':
        # layer 1
        U_6(param1, wires = [0,1])
        U_6(param2, wires = [2,3])
        U_6(param3, wires = [4,5])
        U_6(param4, wires = [6,7])
        # layer 2
        U_6(param5, wires = [1,3])
        U_6(param6, wires = [5,7])
        # layer 3
        U_6(param7, wires = [3,7])
    elif U == 'U_13':
        # layer 1
        U_13(param1, wires = [0,1])
        U_13(param2, wires = [2,3])
        U_13(param3, wires = [4,5])
        U_13(param4, wires = [6,7])
        # layer 2
        U_13(param5, wires = [1,3])
        U_13(param6, wires = [5,7])
        # layer 3
        U_13(param7, wires = [3,7])
    elif U == 'U_14':
        # layer 1
        U_14(param1, wires = [0,1])
        U_14(param2, wires = [2,3])
        U_14(param3, wires = [4,5])
        U_14(param4, wires = [6,7])
        # layer 2
        U_14(param5, wires = [1,3])
        U_14(param6, wires = [5,7])
        # layer 3
        U_14(param7, wires = [3,7])
    elif U == 'U_15':
        # layer 1
        U_15(param1, wires = [0,1])
        U_15(param2, wires = [2,3])
        U_15(param3, wires = [4,5])
        U_15(param4, wires = [6,7])
        # layer 2
        U_15(param5, wires = [1,3])
        U_15(param6, wires = [5,7])
        # layer 3
        U_15(param7, wires = [3,7])
    elif U == 'U_SO4':
        # layer 1
        U_SO4(param1, wires = [0,1])
        U_SO4(param2, wires = [2,3])
        U_SO4(param3, wires = [4,5])
        U_SO4(param4, wires = [6,7])
        # layer 2
        U_SO4(param5, wires = [1,3])
        U_SO4(param6, wires = [5,7])
        # layer 3
        U_SO4(param7, wires = [3,7])
    

    return qml.expval(qml.PauliZ(7))

# Training Quantum Circuits

In [7]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

def cost(params, X, Y, U, U_params, embedding_type, circuit):
    if circuit == 'QCNN':
        predictions = [QCNN(x, params, U, U_params, embedding_type) for x in X]
    elif circuit == 'Hierarchical':
        predictions = [Hierarchical_classifier(x, params, U, U_params, embedding_type) for x in X]
    
    return square_loss(Y, predictions)

def accuracy_test_binary(predictions, labels):
    acc = 0
    for l,p in zip(labels, predictions):
        if np.abs(l - p) < 1:
            acc = acc + 1
    return acc / len(labels)

def accuracy_test_one_class(predictions, labels):
    acc = 0
    for l,p in zip(labels, predictions):
        if np.abs(l - p) < 0.5:
            acc = acc + 1
    return acc / len(labels)

In [8]:
def circuit_training(X_train, Y_train, U, U_params, embedding_type, circuit):
    if circuit == 'QCNN':
        total_params = U_params * 2 + 2 * 2 + 1
    elif circuit == 'Hierarchical':
        total_params = U_params * 7
        
    params = np.random.randn(total_params)
    steps = 150
    learning_rate = 0.1
    batch_size =25
    opt = qml.NesterovMomentumOptimizer(learning_rate)
    
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        params, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, U, U_params, embedding_type, circuit), params)
        if it % 10 == 0:
            print("iteration: ", it, " cost: ", cost_new)
    return params

### MNIST Data loading and processing

Use PCA and Autoencoder to reduce it into 8 features. We test both one-class classification (labeling 0 and 1) and binary classification (labeling -1 and 1).

In [9]:
import tensorflow as tf
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses

def data_load_and_process(classes = [0,1], feature_reduction = 'resize256', binary = True):
    
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0 #normalize the data
    
    x_train_filter_01 = np.where((y_train == classes[0]) | (y_train == classes[1]))
    x_test_filter_01 = np.where((y_test == classes[0]) | (y_test == classes[1]))
    
    x_train_01, x_test_01 = x_train[x_train_filter_01], x_test[x_test_filter_01]
    y_train_01, y_test_01 = y_train[x_train_filter_01], y_test[x_test_filter_01]
    
    if binary == False:
        y_train_01 = [1 if y ==classes[0] else 0 for y in y_train_01]
        y_test_01 = [1 if y ==classes[0] else 0 for y in y_test_01]
    elif binary == True:
        y_train_01 = [1 if y ==classes[0] else -1 for y in y_train_01]
        y_test_01 = [1 if y ==classes[0] else -1 for y in y_test_01]
        
    
    if feature_reduction == 'resize256':   
        x_train_01 = tf.image.resize(x_train_01[:], (256, 1)).numpy()
        x_test_01 = tf.image.resize(x_test_01[:], (256, 1)).numpy()
        x_train_01, x_test_01 = tf.squeeze(x_train_01), tf.squeeze(x_test_01) 
        return x_train_01, x_test_01, y_train_01, y_test_01
    
    elif feature_reduction == 'pca8':
        x_train_01 = tf.image.resize(x_train_01[:], (784, 1)).numpy()
        x_test_01 = tf.image.resize(x_test_01[:], (784, 1)).numpy()
        x_train_01, x_test_01 = tf.squeeze(x_train_01), tf.squeeze(x_test_01)
        
        pca = PCA(8)
        x_train_01 = pca.fit_transform(x_train_01)
        x_test_01 = pca.transform(x_test_01)
        
        #Rescale for angle embedding
        x_train_01, x_test_01 = (x_train_01 + 10) * (np.pi / 20), (x_test_01 + 10) * (np.pi / 20)
        
        return x_train_01, x_test_01, y_train_01, y_test_01

    elif feature_reduction == 'autoencoder8':
        latent_dim = 8 
        class Autoencoder(Model):
            def __init__(self, latent_dim):
                super(Autoencoder, self).__init__()
                self.latent_dim = latent_dim   
                self.encoder = tf.keras.Sequential([
                layers.Flatten(),
                  layers.Dense(latent_dim, activation='relu'),
                ])
                self.decoder = tf.keras.Sequential([
                layers.Dense(784, activation='sigmoid'),
                layers.Reshape((28, 28))
                ])
            def call(self, x):
                encoded = self.encoder(x)
                decoded = self.decoder(encoded)
                return decoded
        
        autoencoder = Autoencoder(latent_dim)
        
        autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
        autoencoder.fit(x_train_01, x_train_01,
                epochs=10,
                shuffle=True,
                validation_data=(x_test_01, x_test_01))
        
        x_train_01, x_test_01 = autoencoder.encoder(x_train_01).numpy(), autoencoder.encoder(x_test_01).numpy()
        #Rescale for Angle Embedding
        x_train_01, x_test_01 = x_train_01 * (np.pi / 50), x_test_01 * (np.pi / 50)
        
        return x_train_01, x_test_01, y_train_01, y_test_01


# Results

Benchmarking function trains and performs accuracy tests for all the given unitary ansatze, feature reduction methods, and circuit structures.

In [10]:
def Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = True):
    I = len(Unitaries)
    J = len(Encodings)
    All_predictions = []

    for i in range(I):
        for j in range(J):
            U = Unitaries[i]
            U_params = U_num_params[i]
            Encoding = Encodings[j]
            if Encoding == 'resize256':
                Embedding = 'Amplitude'
                X_train, X_test, Y_train, Y_test = data_load_and_process(classes = classes, feature_reduction = 'resize256', binary = binary)
            elif Encoding == 'pca8':
                Embedding = 'Angle'
                X_train, X_test, Y_train, Y_test = data_load_and_process(classes = classes, feature_reduction = 'pca8', binary = binary)
            elif Encoding == 'autoencoder8':
                Embedding = 'Angle'
                X_train, X_test, Y_train, Y_test = data_load_and_process(classes = classes, feature_reduction = 'autoencoder8', binary = binary)
        
            print("\n")
            print("Loss History for " + circuit + " circuits, "+ U + " " + Encoding)
            trained_params = circuit_training(X_train, Y_train, U, U_params, Embedding, circuit)
            
            if circuit == 'QCNN':
                predictions = [QCNN(x, trained_params, U, U_params, Embedding) for x in X_test]
            elif circuit == 'Hierarchical':
                predictions = [Hierarchical_classifier(x, trained_params, U, U_params, Embedding) for x in X_test]
                
                
            if binary == True:
                accuracy = accuracy_test_binary(predictions, Y_test)
            elif binary == False:
                accuracy = accuracy_test_one_class(predictions, Y_test)
                
            print("Accuracy for " + U + " " + Encoding + " :" + str(accuracy))

### 1. Results for QCNN circuit

#### 1.1 Binary Classification with 1, -1 labels

In [ ]:
Unitaries = ['U_TTN', 'U_5', 'U_6', 'U_9', 'U_13', 'U_14', 'U_15', 'U_SO4'] 
U_num_params = [2, 10, 10, 4, 6, 6, 4, 6]
Encodings = ['resize256', 'pca8', 'autoencoder8']
classes = [0,1]
circuit = 'QCNN'

In [ ]:
Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = True)

#### 1.2 One Class Classification with labels 0,1

In [ ]:
Unitaries = ['U_TTN', 'U_5', 'U_6', 'U_9', 'U_13', 'U_14', 'U_15', 'U_SO4']
U_num_params = [2, 10, 10, 4, 6, 6, 4, 6]
Encodings = ['resize256', 'pca8', 'autoencoder8']
classes = [0,1]
circuit = 'QCNN'

In [ ]:
Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = False)

## 2. Results for Hierarchical Classifier circuit

#### 2.1 Binary Classification with 1, -1 labeling

In [11]:
Unitaries = ['U_TTN', 'U_5', 'U_6', 'U_13', 'U_14', 'U_15', 'U_SO4'] 
U_num_params = [2, 10, 10, 6, 6, 4, 6]
Encodings = ['resize256', 'pca8', 'autoencoder8']
classes = [0,1]
circuit = 'Hierarchical'

In [12]:
Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = True)



Loss History for Hierarchical circuits, U_TTN resize256
iteration:  0  cost:  1.4254236737289714
iteration:  10  cost:  0.6964376332391631
iteration:  20  cost:  0.385839703052776
iteration:  30  cost:  0.6624269072028788
iteration:  40  cost:  0.4736939711273403
iteration:  50  cost:  0.41880031707818666
iteration:  60  cost:  0.5528576465503573
iteration:  70  cost:  0.5575690099322155
iteration:  80  cost:  0.3721928613545859
iteration:  90  cost:  0.32498144777837257
iteration:  100  cost:  0.19269116603381953
iteration:  110  cost:  0.36746825596288313
iteration:  120  cost:  0.3067873718541184
iteration:  130  cost:  0.3259691446748894
iteration:  140  cost:  0.3607067568495287
Accuracy for U_TTN resize256 :0.9697399527186761


Loss History for Hierarchical circuits, U_TTN pca8
iteration:  0  cost:  1.0644402758747809
iteration:  10  cost:  0.954259410192369
iteration:  20  cost:  0.8311068824305157
iteration:  30  cost:  0.41639199980415315
iteration:  40  cost:  0.40324249508


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.





Loss History for Hierarchical circuits, U_TTN autoencoder8
iteration:  0  cost:  0.9287996620420949
iteration:  10  cost:  0.5839446308103682
iteration:  20  cost:  0.4868617056559588
iteration:  30  cost:  0.46903295184498345
iteration:  40  cost:  0.44547337338514886
iteration:  50  cost:  0.4709291304842248
iteration:  60  cost:  0.2568436782364198
iteration:  70  cost:  0.3363763128664132
iteration:  80  cost:  0.2613623383974762
iteration:  90  cost:  0.36491087001478767
iteration:  100  cost:  0.4185086391128328
iteration:  110  cost:  0.4105554166711026
iteration:  120  cost:  0.4367245350130077
iteration:  130  cost:  0.4112624294409452
iteration:  140  cost:  0.3000756431729917
Accuracy for U_TTN autoencoder8 :0.9144208037825059


Loss History for Hierarchical circuits, U_5 resize256
iteration:  0  cost:  1.0093771219979053
iteration:  10  cost:  0.9999621853491196
iteration:  20  cost:  1.0101142400923957
iteration:  30  cost:  0.9972236499720948
iteration:  40  cost:  0.99


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.





Loss History for Hierarchical circuits, U_5 autoencoder8
iteration:  0  cost:  1.5728436140881967
iteration:  10  cost:  0.7661743282855591
iteration:  20  cost:  0.38780790130099535
iteration:  30  cost:  0.33830022334738397
iteration:  40  cost:  0.3539911278018992
iteration:  50  cost:  0.45580262441743374
iteration:  60  cost:  0.37292889336763885
iteration:  70  cost:  0.37338094469555505
iteration:  80  cost:  0.23451038056745804
iteration:  90  cost:  0.44229134524604924
iteration:  100  cost:  0.3310444440364451
iteration:  110  cost:  0.3337666490473295
iteration:  120  cost:  0.4095344222263811
iteration:  130  cost:  0.47124319364785644
iteration:  140  cost:  0.25162790061853535
Accuracy for U_5 autoencoder8 :0.9598108747044918


Loss History for Hierarchical circuits, U_6 resize256
iteration:  0  cost:  1.642112060648264
iteration:  10  cost:  0.830944947464337
iteration:  20  cost:  0.4198823586796053
iteration:  30  cost:  0.31097897935061325
iteration:  40  cost:  0.2

...



Loss History for Hierarchical circuits, U_14 autoencoder8
iteration:  0  cost:  1.0531902628499
iteration:  10  cost:  0.653871344919674
iteration:  20  cost:  0.4549694961656165
iteration:  30  cost:  0.3529477730389191
iteration:  40  cost:  0.3446139090930156
iteration:  50  cost:  0.37568830368883327
iteration:  60  cost:  0.45092270628133313
iteration:  70  cost:  0.3003136907810043
iteration:  80  cost:  0.4319799455267038
iteration:  90  cost:  0.30556104188015093
iteration:  100  cost:  0.4502572878125459
iteration:  110  cost:  0.313153314063631
iteration:  120  cost:  0.29198377502465633
iteration:  130  cost:  0.2793343526724168
iteration:  140  cost:  0.38240871633668616
Accuracy for U_14 autoencoder8 :0.9574468085106383


Loss History for Hierarchical circuits, U_15 resize256
iteration:  0  cost:  1.2208044008485568
iteration:  10  cost:  0.7338955050518049
iteration:  20  cost:  0.4108902924073049
iteration:  30  cost:  0.38035968692524025
iteration:  40  cost:  0.30557


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.





Loss History for Hierarchical circuits, U_15 autoencoder8
iteration:  0  cost:  0.8872651859536691
iteration:  10  cost:  0.4147219425497003
iteration:  20  cost:  0.2935554253485046
iteration:  30  cost:  0.23305305478436045
iteration:  40  cost:  0.2814739923279155
iteration:  50  cost:  0.19573227856001085
iteration:  60  cost:  0.25801602015058756
iteration:  70  cost:  0.2324566433532428
iteration:  80  cost:  0.21113197789812355
iteration:  90  cost:  0.21007567320295514
iteration:  100  cost:  0.13864058359296139
iteration:  110  cost:  0.199714917508006
iteration:  120  cost:  0.28455062418058696
iteration:  130  cost:  0.2291350930501233
iteration:  140  cost:  0.2884806983329339
Accuracy for U_15 autoencoder8 :0.9626477541371158


Loss History for Hierarchical circuits, U_SO4 resize256
iteration:  0  cost:  1.0171822854323356
iteration:  10  cost:  0.8298700202520206
iteration:  20  cost:  0.3519340983544786
iteration:  30  cost:  0.3380694349477043
iteration:  40  cost:  0


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.





Loss History for Hierarchical circuits, U_SO4 autoencoder8
iteration:  0  cost:  1.1874181132762482
iteration:  10  cost:  0.6734521533917834
iteration:  20  cost:  0.16991175271142292
iteration:  30  cost:  0.24381507497617305
iteration:  40  cost:  0.2902567195381991
iteration:  50  cost:  0.13763112489834425
iteration:  60  cost:  0.503709015727615
iteration:  70  cost:  0.2172640387428687
iteration:  80  cost:  0.2533184918919411
iteration:  90  cost:  0.4198843568834553
iteration:  100  cost:  0.1641041159778238
iteration:  110  cost:  0.14945240105600627
iteration:  120  cost:  0.262197695098777
iteration:  130  cost:  0.22213564298954314
iteration:  140  cost:  0.35904066271442786
Accuracy for U_SO4 autoencoder8 :0.9536643026004729


#### 2.2 One Class Classification with 0, 1 labeling

In [13]:
Unitaries = ['U_TTN', 'U_5', 'U_6', 'U_13', 'U_14', 'U_15', 'U_SO4'] 
U_num_params = [2, 10, 10, 6, 6, 4, 6]
Encodings = ['resize256', 'pca8', 'autoencoder8']
classes = [0,1]
circuit = 'Hierarchical'

In [14]:
Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = False)



Loss History for Hierarchical circuits, U_TTN resize256
iteration:  0  cost:  0.35525773742240896
iteration:  10  cost:  0.3328691558220538
iteration:  20  cost:  0.2140754008832847
iteration:  30  cost:  0.1015911831206335
iteration:  40  cost:  0.08523465681568673
iteration:  50  cost:  0.13504519589169214
iteration:  60  cost:  0.16959733491187542
iteration:  70  cost:  0.2056834277536673
iteration:  80  cost:  0.10870341952284272
iteration:  90  cost:  0.11761417128397657
iteration:  100  cost:  0.11893951068590494
iteration:  110  cost:  0.09069035817993487
iteration:  120  cost:  0.13341179349553453
iteration:  130  cost:  0.08403569285131886
iteration:  140  cost:  0.10402214866940401
Accuracy for U_TTN resize256 :0.9234042553191489


Loss History for Hierarchical circuits, U_TTN pca8
iteration:  0  cost:  0.4030474611440642
iteration:  10  cost:  0.2398302903707779
iteration:  20  cost:  0.19140073583434347
iteration:  30  cost:  0.2604377151220624
iteration:  40  cost:  0.17


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.





Loss History for Hierarchical circuits, U_TTN autoencoder8
iteration:  0  cost:  0.45104668114502133
iteration:  10  cost:  0.13006874763709894
iteration:  20  cost:  0.16286208703665153
iteration:  30  cost:  0.06777652117910357
iteration:  40  cost:  0.0459391601608008
iteration:  50  cost:  0.06394366327047958
iteration:  60  cost:  0.03721093113572069
iteration:  70  cost:  0.034127685274360886
iteration:  80  cost:  0.08091913406808816
iteration:  90  cost:  0.05133387049661154
iteration:  100  cost:  0.039230947766204144
iteration:  110  cost:  0.07935346013945002
iteration:  120  cost:  0.05189745380239168
iteration:  130  cost:  0.05803668770364166
iteration:  140  cost:  0.03936430964616579
Accuracy for U_TTN autoencoder8 :0.9687943262411347


Loss History for Hierarchical circuits, U_5 resize256
iteration:  0  cost:  0.997476893650433
iteration:  10  cost:  0.4954167402847351
iteration:  20  cost:  0.24272056847158224
iteration:  30  cost:  0.20005502694858276
iteration:  4


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.





Loss History for Hierarchical circuits, U_5 autoencoder8
iteration:  0  cost:  0.792473784379411
iteration:  10  cost:  0.14829486428336727
iteration:  20  cost:  0.09960659126649281
iteration:  30  cost:  0.07587366534578206
iteration:  40  cost:  0.13129787976785048
iteration:  50  cost:  0.170075390009835
iteration:  60  cost:  0.25372058195333985
iteration:  70  cost:  0.08029172844577716
iteration:  80  cost:  0.133766905830799
iteration:  90  cost:  0.11004888054169273
iteration:  100  cost:  0.1343455148924195
iteration:  110  cost:  0.08182072952662356
iteration:  120  cost:  0.0937487055403106
iteration:  130  cost:  0.1531691639080851
iteration:  140  cost:  0.15391712490631615
Accuracy for U_5 autoencoder8 :0.8770685579196218


Loss History for Hierarchical circuits, U_6 resize256
iteration:  0  cost:  0.28987397936802606
iteration:  10  cost:  0.24476557047361036
iteration:  20  cost:  0.221873622868841
iteration:  30  cost:  0.20406692830207832
iteration:  40  cost:  0.1

...



Loss History for Hierarchical circuits, U_14 autoencoder8
iteration:  0  cost:  0.3572586296063401
iteration:  10  cost:  0.19502081378433278
iteration:  20  cost:  0.20977013776562026
iteration:  30  cost:  0.21195527335313763
iteration:  40  cost:  0.11075903707560085
iteration:  50  cost:  0.13008799116386732
iteration:  60  cost:  0.11451330637085343
iteration:  70  cost:  0.10083753816851596
iteration:  80  cost:  0.1093915683024726
iteration:  90  cost:  0.10989801530048707
iteration:  100  cost:  0.09837303827764397
iteration:  110  cost:  0.1121862820059631
iteration:  120  cost:  0.0801396764777206
iteration:  130  cost:  0.08387410004447958
iteration:  140  cost:  0.07184851728393073
Accuracy for U_14 autoencoder8 :0.9706855791962175


Loss History for Hierarchical circuits, U_15 resize256
iteration:  0  cost:  0.5960445067441198
iteration:  10  cost:  0.1463659383104196
iteration:  20  cost:  0.13632372659031866
iteration:  30  cost:  0.10195681024820946
iteration:  40  co


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.





Loss History for Hierarchical circuits, U_15 autoencoder8
iteration:  0  cost:  0.1607200299821103
iteration:  10  cost:  0.10707737984189969
iteration:  20  cost:  0.04105760161061437
iteration:  30  cost:  0.04436563552708688
iteration:  40  cost:  0.029634263644470325
iteration:  50  cost:  0.030843134844376664
iteration:  60  cost:  0.030263129330024053
iteration:  70  cost:  0.02616273426498309
iteration:  80  cost:  0.014233070226215158
iteration:  90  cost:  0.025472571026347094
iteration:  100  cost:  0.05569407544679674
iteration:  110  cost:  0.019367488019393107
iteration:  120  cost:  0.02061968137896745
iteration:  130  cost:  0.010410629662882436
iteration:  140  cost:  0.03161753524011306
Accuracy for U_15 autoencoder8 :0.992434988179669


Loss History for Hierarchical circuits, U_SO4 resize256
iteration:  0  cost:  0.3553435337738612
iteration:  10  cost:  0.3320599358207073
iteration:  20  cost:  0.2607467730973488
iteration:  30  cost:  0.2345352265007858
iteration:


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.





Loss History for Hierarchical circuits, U_SO4 autoencoder8
iteration:  0  cost:  0.3510546701533224
iteration:  10  cost:  0.09761463420025823
iteration:  20  cost:  0.02397880549625883
iteration:  30  cost:  0.013904913720665116
iteration:  40  cost:  0.019979492217430412
iteration:  50  cost:  0.05444298196984325
iteration:  60  cost:  0.02591592070788897
iteration:  70  cost:  0.03392110830378884
iteration:  80  cost:  0.018468827989503687
iteration:  90  cost:  0.03045450012979467
iteration:  100  cost:  0.01612231112560783
iteration:  110  cost:  0.01679955287509967
iteration:  120  cost:  0.034568265798485064
iteration:  130  cost:  0.011824425365508871
iteration:  140  cost:  0.02467169435996436
Accuracy for U_SO4 autoencoder8 :0.9877068557919622
